In [6]:
import pandas as pd
order_item_wide = pd.read_csv("../raw/order_item_wide_gmv_analysis2.csv")

# 8.读取产品详情表
products_detail = pd.read_csv("../raw/olist_products_dataset.csv")

# 9.空值检查
print(f"列缺失情况汇总:\n{products_detail.isnull().sum()}")
products_detail['product_category_name'] = products_detail['product_category_name'].fillna("unknown")

# 10.筛选要使用字段
products = products_detail.loc[:,['product_id','product_category_name']]

列缺失情况汇总:
product_id                      0
product_category_name         610
product_name_lenght           610
product_description_lenght    610
product_photos_qty            610
product_weight_g                2
product_length_cm               2
product_height_cm               2
product_width_cm                2
dtype: int64


In [7]:
# 11.关联完整性检查
missing_category = order_item_wide[~order_item_wide['product_id'].isin(products['product_id'])]
print("\n products中找不到对应类目的数量：", len(missing_category)) # 输出0

# 12.合并两表
item_category = order_item_wide.merge(products_detail,how='inner',on = 'product_id')


 products中找不到对应类目的数量： 0


In [8]:
# 13.品类分析
category_counts = item_category['product_category_name'].value_counts()

In [ ]:

category_agg = (
    item_category.groupby('product_category_name', as_index=False)
    .agg({
        'price':'sum',
        'freight_value':'sum'
    })
)

# 修正括号问题并计算总GMV
category_agg['gmv_value'] = category_agg['price'] + category_agg['freight_value']
total_gmv = category_agg['gmv_value'].sum()

# 5. 计算占比 + 累积占比
category_agg = category_agg.sort_values('gmv_value', ascending=False).reset_index(drop=True)
category_agg['gmv_share'] = category_agg['gmv_value'] / total_gmv
category_agg['cum_share'] = category_agg['gmv_share'].cumsum()

# category_agg.to_csv("../processed/category_agg.csv")

def classify(row):
    if row['cum_share'] <= 0.8:
        return row['product_category_name']
    elif row['cum_share'] <= 0.95:
        return 'mid_others'
    else:
        return 'long_tail'

category_agg['category_final'] = category_agg.apply(classify, axis=1)
category_agg.loc[category_agg['product_category_name'] == 'unknown', 'category_final'] = 'unknown'

# merge 回订单明细
item_category = item_category.merge(
    category_agg[['product_category_name','category_final']],
    on='product_category_name',
    how='left'
)

item_category = item_category.loc[:,['order_id','product_id','category_final','order_purchase_timestamp','price','freight_value']]

# item_category.to_csv("../processed/category_gmv.csv")
